# FABRIC Resource Finder

Query FABRIC testbed sites for available resources.

## Features:
- Find sites with specific NICs
- Find sites with GPUs, FPGAs, NVMe
- Filter by multiple criteria
- Export results to various formats

## 1. Setup

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
    print("✅ Fablib initialized successfully")
    fablib.show_config()
except Exception as e:
    print(f"❌ Exception: {e}")
    

## 2. Helper Function - Get Sites DataFrame

In [ ]:
import pandas as pd

def get_sites_dataframe(fablib):
    """
    Get all FABRIC sites with their resources as a DataFrame.
    
    Args:
        fablib: FablibManager instance
        
    Returns:
        pandas.DataFrame with site information and resources
    """
    resources = fablib.get_resources()
    sites_list = []

    for site_name in resources.sites:
        site = resources.sites[site_name]
        
        # Build site dictionary
        site_dict = {'name': site_name}
        
        # Basic attributes
        try:
            site_dict['state'] = site.get_state()
        except:
            site_dict['state'] = None
        
        # Get location info
        try:
            location = site.get_location_postal()
            site_dict['address'] = location
        except:
            site_dict['address'] = None
        
        # Get basic resources
        try:
            site_dict['cores_available'] = site.get_core_available()
            site_dict['cores_capacity'] = site.get_core_capacity()
            site_dict['cores_allocated'] = site.get_core_allocated()
            
            site_dict['ram_available'] = site.get_ram_available()
            site_dict['ram_capacity'] = site.get_ram_capacity()
            site_dict['ram_allocated'] = site.get_ram_allocated()
            
            site_dict['disk_available'] = site.get_disk_available()
            site_dict['disk_capacity'] = site.get_disk_capacity()
            site_dict['disk_allocated'] = site.get_disk_allocated()
        except:
            pass
        
        # Get component info from site_info (NICs, GPUs, FPGAs, NVMe, etc.)
        try:
            site_info = site.site_info
            for component_name, component_data in site_info.items():
                if isinstance(component_data, dict) and 'capacity' in component_data:
                    # Create columns for each component
                    site_dict[f'{component_name}_capacity'] = component_data.get('capacity', 0)
                    site_dict[f'{component_name}_allocated'] = component_data.get('allocated', 0)
                    site_dict[f'{component_name}_available'] = (
                        component_data.get('capacity', 0) - component_data.get('allocated', 0)
                    )
        except Exception as e:
            pass
        
        sites_list.append(site_dict)

    return pd.DataFrame(sites_list)


## 3. Quick Query - Sites with SmartNIC ConnectX-5

In [ ]:
print("🔎 Sites with SmartNIC ConnectX-5 available:\n")

sites_df = get_sites_dataframe(fablib)
cx5_sites = sites_df[sites_df['smartnic-connectx-5_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'smartnic-connectx-5_available']
print(cx5_sites[display_cols].to_string(index=False))


## 4. Quick Query - Sites with SmartNIC ConnectX-6

In [ ]:
print("🔎 Sites with SmartNIC ConnectX-6 available:\n")

sites_df = get_sites_dataframe(fablib)
cx6_sites = sites_df[sites_df['smartnic-connectx-6_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'smartnic-connectx-6_available']
print(cx6_sites[display_cols].to_string(index=False))


## 5. Quick Query - Sites with SharedNIC ConnectX-6

In [ ]:
print("🔎 Sites with SharedNIC ConnectX-6 available:\n")

sites_df = get_sites_dataframe(fablib)
shared_cx6_sites = sites_df[sites_df['sharednic-connectx-6_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'sharednic-connectx-6_available']
print(shared_cx6_sites[display_cols].to_string(index=False))


## 6. Quick Query - Sites with DPU NICs (ConnectX-7 100G)

In [ ]:
print("🔎 Sites with DPU NICs (ConnectX-7 100G) available:\n")

sites_df = get_sites_dataframe(fablib)
dpu_sites = sites_df[sites_df['smartnic-connectx-7-100_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'smartnic-connectx-7-100_available']
print(dpu_sites[display_cols].to_string(index=False))


## 7. Quick Query - Sites with BOTH SmartNIC ConnectX-5 and ConnectX-6

In [ ]:
print("🔎 Sites with both SmartNIC ConnectX-5 and ConnectX-6:\n")

sites_df = get_sites_dataframe(fablib)
both_nics_sites = sites_df[
    (sites_df['smartnic-connectx-5_available'].fillna(0) > 0) & 
    (sites_df['smartnic-connectx-6_available'].fillna(0) > 0)
]

display_cols = ['name', 'address', 'smartnic-connectx-5_available', 'smartnic-connectx-6_available']
print(both_nics_sites[display_cols].to_string(index=False))


## 8. Advanced Query - Sites with GPUs

In [ ]:
print("🔎 Sites with RTX6000 GPUs:\n")

sites_df = get_sites_dataframe(fablib)
rtx6000_sites = sites_df[sites_df['gpu-rtx6000_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'gpu-rtx6000_available']
print(rtx6000_sites[display_cols].to_string(index=False))

print("\n🔎 Sites with Tesla T4 GPUs:\n")

t4_sites = sites_df[sites_df['gpu-tesla t4_available'].fillna(0) > 0]
display_cols = ['name', 'address', 'gpu-tesla t4_available']
print(t4_sites[display_cols].to_string(index=False))

print("\n🔎 Sites with A30 GPUs:\n")

a30_sites = sites_df[sites_df['gpu-a30_available'].fillna(0) > 0]
display_cols = ['name', 'address', 'gpu-a30_available']
print(a30_sites[display_cols].to_string(index=False))


## 9. Advanced Query - Sites with FPGAs

In [ ]:
print("🔎 Sites with Xilinx U280 FPGAs:\n")

sites_df = get_sites_dataframe(fablib)
fpga_sites = sites_df[sites_df['fpga-xilinx-u280_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'fpga-xilinx-u280_available']
print(fpga_sites[display_cols].to_string(index=False))


## 10. Advanced Query - Sites with NVMe Storage

In [ ]:
print("🔎 Sites with NVMe storage:\n")

sites_df = get_sites_dataframe(fablib)
nvme_sites = sites_df[sites_df['nvme-p4510_available'].fillna(0) > 0]

display_cols = ['name', 'address', 'nvme-p4510_available']
print(nvme_sites[display_cols].to_string(index=False))


## 11. Custom Resource Finder Function

In [ ]:
def find_sites_with_resources(
    fablib,
    min_cores=0,
    min_ram=0,
    min_disk=0,
    smartnic_connectx_5=0,
    smartnic_connectx_6=0,
    smartnic_connectx_7_100=0,
    smartnic_connectx_7_400=0,
    sharednic_connectx_6=0,
    gpu_rtx6000=0,
    gpu_a30=0,
    gpu_a40=0,
    gpu_tesla_t4=0,
    fpga_u280=0,
    nvme=0,
    verbose=True
):
    """
    Find FABRIC sites matching resource criteria.
    
    Args:
        fablib: FablibManager instance
        min_cores: Minimum CPU cores available
        min_ram: Minimum RAM in GB
        min_disk: Minimum disk in GB
        smartnic_connectx_5: Minimum ConnectX-5 SmartNICs
        smartnic_connectx_6: Minimum ConnectX-6 SmartNICs
        smartnic_connectx_7_100: Minimum ConnectX-7 100G SmartNICs
        smartnic_connectx_7_400: Minimum ConnectX-7 400G SmartNICs
        sharednic_connectx_6: Minimum ConnectX-6 SharedNICs
        gpu_rtx6000: Minimum RTX6000 GPUs
        gpu_a30: Minimum A30 GPUs
        gpu_a40: Minimum A40 GPUs
        gpu_tesla_t4: Minimum Tesla T4 GPUs
        fpga_u280: Minimum U280 FPGAs
        nvme: Minimum NVMe devices
        verbose: Print detailed info
    
    Returns:
        DataFrame of matching sites
    """
    sites_df = get_sites_dataframe(fablib)
    
    # Helper function to safely get numeric value, treating NaN/None as 0
    def safe_get(row, col, default=0):
        val = row.get(col, default)
        if pd.isna(val):
            return default
        return val
    
    def filter_func(row):
        return (
            safe_get(row, 'cores_available') >= min_cores and
            safe_get(row, 'ram_available') >= min_ram and
            safe_get(row, 'disk_available') >= min_disk and
            safe_get(row, 'smartnic-connectx-5_available') >= smartnic_connectx_5 and
            safe_get(row, 'smartnic-connectx-6_available') >= smartnic_connectx_6 and
            safe_get(row, 'smartnic-connectx-7-100_available') >= smartnic_connectx_7_100 and
            safe_get(row, 'smartnic-connectx-7-400_available') >= smartnic_connectx_7_400 and
            safe_get(row, 'sharednic-connectx-6_available') >= sharednic_connectx_6 and
            safe_get(row, 'gpu-rtx6000_available') >= gpu_rtx6000 and
            safe_get(row, 'gpu-a30_available') >= gpu_a30 and
            safe_get(row, 'gpu-a40_available') >= gpu_a40 and
            safe_get(row, 'gpu-tesla t4_available') >= gpu_tesla_t4 and
            safe_get(row, 'fpga-xilinx-u280_available') >= fpga_u280 and
            safe_get(row, 'nvme-p4510_available') >= nvme
        )
    
    if verbose:
        print("🔎 Searching for sites with:")
        if min_cores: print(f"   • Cores: >= {min_cores}")
        if min_ram: print(f"   • RAM: >= {min_ram} GB")
        if min_disk: print(f"   • Disk: >= {min_disk} GB")
        if smartnic_connectx_5: print(f"   • SmartNIC ConnectX-5: >= {smartnic_connectx_5}")
        if smartnic_connectx_6: print(f"   • SmartNIC ConnectX-6: >= {smartnic_connectx_6}")
        if smartnic_connectx_7_100: print(f"   • SmartNIC ConnectX-7 100G: >= {smartnic_connectx_7_100}")
        if smartnic_connectx_7_400: print(f"   • SmartNIC ConnectX-7 400G: >= {smartnic_connectx_7_400}")
        if sharednic_connectx_6: print(f"   • SharedNIC ConnectX-6: >= {sharednic_connectx_6}")
        if gpu_rtx6000: print(f"   • RTX6000: >= {gpu_rtx6000}")
        if gpu_a30: print(f"   • A30: >= {gpu_a30}")
        if gpu_a40: print(f"   • A40: >= {gpu_a40}")
        if gpu_tesla_t4: print(f"   • Tesla T4: >= {gpu_tesla_t4}")
        if fpga_u280: print(f"   • U280 FPGA: >= {fpga_u280}")
        if nvme: print(f"   • NVMe: >= {nvme}")
        print()
    
    # Apply filter
    mask = sites_df.apply(filter_func, axis=1)
    result_df = sites_df[mask]
    
    # Select relevant columns for display
    display_cols = ['name', 'state', 'address']
    if min_cores: display_cols.append('cores_available')
    if min_ram: display_cols.append('ram_available')
    if min_disk: display_cols.append('disk_available')
    if smartnic_connectx_5: display_cols.append('smartnic-connectx-5_available')
    if smartnic_connectx_6: display_cols.append('smartnic-connectx-6_available')
    if smartnic_connectx_7_100: display_cols.append('smartnic-connectx-7-100_available')
    if smartnic_connectx_7_400: display_cols.append('smartnic-connectx-7-400_available')
    if sharednic_connectx_6: display_cols.append('sharednic-connectx-6_available')
    if gpu_rtx6000: display_cols.append('gpu-rtx6000_available')
    if gpu_a30: display_cols.append('gpu-a30_available')
    if gpu_a40: display_cols.append('gpu-a40_available')
    if gpu_tesla_t4: display_cols.append('gpu-tesla t4_available')
    if fpga_u280: display_cols.append('fpga-xilinx-u280_available')
    if nvme: display_cols.append('nvme-p4510_available')
    
    # Only show columns that exist
    display_cols = [col for col in display_cols if col in result_df.columns]
    
    print(f"✅ Found {len(result_df)} matching sites:\n")
    if len(result_df) > 0:
        print(result_df[display_cols].to_string(index=False))
    else:
        print("No sites match the specified criteria.")
    
    return result_df

# Example: Find sites with at least 8 cores, 32GB RAM, and SharedNIC ConnectX-6
find_sites_with_resources(
    fablib,
    min_cores=8,
    min_ram=32,
    sharednic_connectx_6=1
)


## 12. Find Best Site for Your Topology

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

def find_best_site_for_topology(fablib, topology):
    """
    Find best FABRIC site for a given topology.
    
    Args:
        fablib: FablibManager instance
        topology: SiteTopology model or dict
    
    Returns:
        DataFrame of suitable sites
    """
    from slice_utils_models import load_topology_from_dict
    
    # Convert to model if dict
    if isinstance(topology, dict):
        topology = load_topology_from_dict(topology)
    
    # Analyze resource requirements
    total_cores = 0
    total_ram = 0
    total_disk = 0
    needs_smartnic_cx5 = 0
    needs_smartnic_cx6 = 0
    needs_sharednic_cx6 = 0
    needs_gpu = 0
    needs_fpga = 0
    needs_nvme = 0
    
    for node in topology.site_topology_nodes.iter_nodes():
        total_cores += node.capacity.cpu
        total_ram += node.capacity.ram
        total_disk += node.capacity.disk
        
        # Count NICs
        for nic in node.pci.network.values():
            model = nic.model.lower()
            if 'connectx-5' in model or 'connectx_5' in model:
                needs_smartnic_cx5 += 1
            elif 'connectx-6' in model or 'connectx_6' in model:
                if 'shared' in model:
                    needs_sharednic_cx6 += 1
                else:
                    needs_smartnic_cx6 += 1
        
        # Count other hardware
        needs_gpu += len(node.pci.gpu)
        needs_fpga += len(node.pci.fpga)
        needs_nvme += len(node.pci.nvme)
    
    print("📊 Topology Resource Requirements:")
    print(f"   • Total Cores: {total_cores}")
    print(f"   • Total RAM: {total_ram} GB")
    print(f"   • Total Disk: {total_disk} GB")
    if needs_smartnic_cx5: print(f"   • SmartNIC ConnectX-5: {needs_smartnic_cx5}")
    if needs_smartnic_cx6: print(f"   • SmartNIC ConnectX-6: {needs_smartnic_cx6}")
    if needs_sharednic_cx6: print(f"   • SharedNIC ConnectX-6: {needs_sharednic_cx6}")
    if needs_gpu: print(f"   • GPUs: {needs_gpu}")
    if needs_fpga: print(f"   • FPGAs: {needs_fpga}")
    if needs_nvme: print(f"   • NVMe: {needs_nvme}")
    print()
    
    # Find matching sites
    return find_sites_with_resources(
        fablib,
        min_cores=total_cores,
        min_ram=total_ram,
        min_disk=total_disk,
        smartnic_connectx_5=needs_smartnic_cx5,
        smartnic_connectx_6=needs_smartnic_cx6,
        sharednic_connectx_6=needs_sharednic_cx6,
        gpu_rtx6000=needs_gpu,  # Note: May need to adjust based on GPU type
        fpga_u280=needs_fpga,
        nvme=needs_nvme,
        verbose=False
    )



In [ ]:
import sys
from pathlib import Path

# Add parent directory to Python path to import modules
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root))

print(f"✅ Python path configured")
print(f"   Repository root: {repo_root}")

# Import modules
from slice_utils_models import load_topology_from_yaml_file, SiteTopology
import slice_deployment as sd
import slice_network_config as snc
import slice_ssh_setup as ssh

print("✅ Modules imported successfully")

# Define YAML directory
YAML_DIR = repo_root / "yaml"
print(f"✅ YAML directory: {YAML_DIR}")


# Site topology YAML file
site_topology_yaml = "../yaml/_slice_topology_7-1.yaml"

# Load and validate topology (raises ValidationError if invalid)
try:
    topology = load_topology_from_yaml_file(site_topology_yaml)
    print("✅ Topology loaded and validated successfully!")
    print(f"   Nodes: {len(topology.site_topology_nodes.nodes)}")
    print(f"   Networks: {len(topology.site_topology_networks.networks)}")
    find_best_site_for_topology(fablib, topology)

except Exception as e:
    print(f"❌ Validation failed: {e}")
    raise





## 13. Export Site Availability to File

In [ ]:
def export_site_availability(fablib, filename="site_availability.txt"):
    """
    Export current site availability to file.
    
    Args:
        fablib: FablibManager instance
        filename: Output filename
    """
    from datetime import datetime
    
    # Get all sites
    sites_df = get_sites_dataframe(fablib)
    
    # Add timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(filename, 'w') as f:
        f.write(f"FABRIC Site Availability Report\n")
        f.write(f"Generated: {timestamp}\n")
        f.write("=" * 80 + "\n\n")
        f.write(sites_df.to_string())
    
    print(f"✅ Exported to: {filename}")

# Uncomment to export
# export_site_availability(fablib)

## 14. Get All Available Resource Types

In [ ]:
# Get all sites as DataFrame
sites_df = get_sites_dataframe(fablib)

# Find columns related to hardware
hardware_columns = [
    col for col in sites_df.columns 
    if any(hw in col.lower() for hw in [
        'nic', 'gpu', 'fpga', 'nvme', 'cores', 'ram', 'disk', 
        'connectx', 'tesla', 'rtx', 'a30', 'a40',
        'u280', 'smartnic', 'sharednic'
    ]) and 'available' in col.lower()
]

print("📋 Available Resource Types in FABRIC:\n")
for col in sorted(hardware_columns):
    if col in sites_df.columns:
        # Handle NaN values properly
        total_available = sites_df[col].fillna(0).sum() if pd.api.types.is_numeric_dtype(sites_df[col]) else "N/A"
        if isinstance(total_available, float):
            total_available = int(total_available)
        print(f"   • {col}: {total_available}")


## 15. Summary and Use Cases

In [ ]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                    FABRIC Resource Finder - Summary
═══════════════════════════════════════════════════════════════════════════════

This notebook provides comprehensive tools for finding FABRIC resources:

📌 Quick Queries:
   • Find sites with specific NICs (ConnectX-5, ConnectX-6, ConnectX-7)
   • Find sites with GPUs (RTX6000, A30, A40, Tesla T4)
   • Find sites with FPGAs (Xilinx U280)
   • Find sites with NVMe storage

📌 Advanced Features:
   • Custom multi-criteria resource finder
   • Automatic site selection for topology
   • Export results to file
   • List all available resource types

📌 Use Cases:
   • Planning: Find suitable sites before deployment
   • Optimization: Choose best site for your workload
   • Monitoring: Track resource availability over time

📌 Key Functions:
   • get_sites_dataframe(fablib) - Get all site data as DataFrame
   • find_sites_with_resources(...) - Find sites matching criteria
   • find_best_site_for_topology(fablib, topology) - Match topology to site
   • export_site_availability(fablib, filename) - Export to file

═══════════════════════════════════════════════════════════════════════════════
""")